# Introduction

___
Name: Fadhiil Dzaki Mulyana<br>
Batch: HCK-22

Notebook ini dibangun untuk uji kualitas data menggunakan Great Expectation (GX).
___

# Libraries

In [1]:
# import lib
from great_expectations.data_context import FileDataContext
import pandas as pd

# Setup

In [2]:
# save report
context = FileDataContext.create(project_root_dir='./')

Data hasil validasi akan tersimpan dalam project root.

# Connect to Data

Sesi ini akan melakukan:
- Membuat datasource baru (csv_inventory_data).
- Menambah asset data csv ke datasource (inventory_data).
- Membuat batch request dari data asset

In [3]:
# define datasource name
datasource_name = 'csv_inventory_data'
datasource = context.sources.add_pandas(datasource_name)

# define data asset
asset_name = 'inventory_data'
data_path = 'P2M3_fadhiil_data_clean.csv'
asset = datasource.add_csv_asset(
    asset_name,
    filepath_or_buffer=data_path,
    parse_dates=['date']
)

# batch request
batch_req = asset.build_batch_request()

# Create Validator

Sesi ini akan melakukan:
- Membangun expectation suite  untuk mendefinisikan standar data.
- Membuat validator untuk memvalidasi data.

In [4]:
# create suite
expect_suite_name = 'expectation_inventory_data'
expect_suite_name = context.add_or_update_expectation_suite(expect_suite_name)

# create validator
validator = context.get_validator(
    batch_request=batch_req,
    expectation_suite=expect_suite_name
)

# check
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,date,store_id,product_id,category,region,inventory_level,units_sold,units_ordered,demand_forecast,price,discount,weather_condition,event,competitor_pricing,seasonality,code
0,2022-01-01,s001,p0001,groceries,north,231,127,55,135.47,33.50,20,rainy,0,29.69,autumn,s001-p0001-220101
1,2022-01-01,s001,p0002,toys,south,204,150,66,144.04,63.01,20,sunny,0,66.16,autumn,s001-p0002-220101
2,2022-01-01,s001,p0003,toys,west,102,65,51,74.02,27.99,10,sunny,1,31.32,summer,s001-p0003-220101
3,2022-01-01,s001,p0004,toys,north,469,61,164,62.18,32.72,10,cloudy,1,34.74,autumn,s001-p0004-220101
4,2022-01-01,s001,p0005,electronics,east,166,14,135,9.26,73.64,0,sunny,0,68.95,summer,s001-p0005-220101


# Expectations

## Expectation 1: To Be Unique

### Code

Memastikan 'code' adalah kolom unique identifier untuk tiap baris sehingga nilainya tidak boleh lebih dari 1.

In [5]:
# code id must unique
validator.expect_column_values_to_be_unique('code')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Kolom 'code' tervalidasi bahwa seluruh nilainya unique.

## Expectation 2: To Be Between min-max value

### Discount

'discount' penjualan berkisar antara 0-100, sehingga nilainya tidak boleh lebih atau kurang dari nilai tersebut.

In [6]:
# discount must be between 0-100
validator.expect_column_values_to_be_between(
    column='discount', 
    min_value=0, 
    max_value=100
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

'discount' pada dataset tervalidasi nilainya ebrkisar 0-100.

### Numeric Columns Can't Bellow Zero

'inventory_level','units_sold','units_ordered','price','competitor_pricing' adalah kolom numerik yang nilainya berkisar dari 0 - tak hingga.

In [7]:
# column cannot less than 0
no_minus = ['inventory_level','units_sold','units_ordered','price','competitor_pricing']

# loop validation that no_minus column cannot less than 0
for i in no_minus:
    # validation per column
    validator.expect_column_values_to_be_between(
        i,
        min_value=0
    )

    # save
    valid = validator.expect_column_values_to_be_between(i, min_value=0)
    # show output succes or not
    print(f"{i} validation: {valid['success']}")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

inventory_level validation: True


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

price validation: True


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

competitor_pricing validation: True


## Expectation 3: To Be In Set

### Weather Condition

Berdasarkan [Met Office](https://www.metoffice.gov.uk/weather/learn-about/weather/types-of-weather), macam-macam cuaca terbagi menjadi: cloudy, snowy, frosty, rainy, foggy, windy, stormy, dan sunny. Memvalidasi 'weather_condition untuk memastikan tidak ada cuaca selain yang telah disebutkan menjadi penting agar kualitas data semakin terjamin.

In [8]:
# ensure weather is real weather
validator.expect_column_values_to_be_in_set(
    'weather_condition',
    ['cloudy', 'snowy', 'frosty', 'rainy', 'foggy', 'windy', 'stormy', 'sunny']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa 'weather_condition' tidak memiliki nilai yang aneh.

### Seasonality

Berdasarkan [NGO](https://education.nationalgeographic.org/resource/season/), musim terbagi menjadi: spring, summer, autumn, dan winter. Memvalidasi 'seasonality' untuk memastikan tidak ada musim selain yang telah disebutkan menjadi penting agar kualitas data semakin terjamin.

In [9]:
# ensure season is 4 season
validator.expect_column_values_to_be_in_set(
    'seasonality',
    ['spring', 'summer', 'autumn', 'winter']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa 'seasonality' tidak memiliki nilai yang aneh.

## Expectation 4: To Be In Type List

### Price & Competitor Price

'price dan 'competitor_pricing' adalah kolom yang berisi mata uang. Agar data tervalidasi kualitasnya, dilakukan validasi untuk memastikan kolom kolom tersebut memiliki tipe data float atau integer.

In [10]:
# price column
prices = ['price', 'competitor_pricing']

# ensure prices are in float
for i in prices:
    validator.expect_column_values_to_be_in_type_list(
        i,
        ['float','integer']
    )
    
    # save in valid
    valid = validator.expect_column_values_to_be_in_type_list(i,['integer','float'])
    
    # show success or not
    print(f"{i} validation: {valid['success']} - {valid['result']['observed_value']}")


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

price validation: True - float64


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

competitor_pricing validation: True - float64


Teravalidasi bahwa 'price' dan 'competitor_price' memiliki tipe data float.

### Date

'date' berisi informasi mengenai tanggal. maka dari itu, perlu untuk memastikan tipe data 'date' adalah datetime.

In [11]:
# ensure date dtype is datetime
validator.expect_column_values_to_be_in_type_list(
    'date',
    ['datetime64[ns]','datetime']
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "datetime64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

'date' tervalidasi memiliki tipe data datetime64.

## Expectation 5: To Be Equal Set

### Event

'event' adalah kolom boolean dengan value 0 dan 1. Validasi dilakukan untuk memastikan data tidak memiliki value selain 0 dan 1.

In [12]:
# ensure event only has 2 value
validator.expect_column_distinct_values_to_equal_set(
    'event',
    [0,1]
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      0,
      1
    ],
    "details": {
      "value_counts": [
        {
          "value": 0,
          "count": 36747
        },
        {
          "value": 1,
          "count": 36353
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa 'event' hanya memiliki value 0 dan 1.

## Expectation 6: to match regex

### Date

'date' memiliki format yyyy-mm-dd. Untuk memvalidasi hal tersebut dilakukan validator dengan ekspektasi 'date' memiliki format tersebut.

In [13]:
# ensure date format is yyyy-mm-dd
validator.expect_column_values_to_match_regex(
    'date',
    regex=r'\d{4}-\d{2}-\d{2}'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa seluruh nilai 'date' memiliki format yyyy=mm=dd.

### Store ID

'store_id' memiliki format s___. Untuk memvalidasi hal tersebut dilakukan validator dengan ekspektasi 'store_id' memiliki format tersebut.

In [14]:
# ensure store id format is s___
validator.expect_column_values_to_match_regex(
    'store_id',
    regex=r's\d{3}'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa seluruh nilai 'store_id' memiliki format s___.

### Product ID

'product_id' memiliki format p____. Untuk memvalidasi hal tersebut dilakukan validator dengan ekspektasi 'product_id' memiliki format p_ersebut.

In [15]:
# ensure product_id format is p____
validator.expect_column_values_to_match_regex(
    'product_id',
    regex=r'p\d{4}'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa seluruh nilai 'product_id' memiliki format p____.

## Expectation 7: Most Common Value

### Weather condition

Secara logika, orang-orang biasanya belanja ketika cuaca sedang cerah, khususnya barang-barang elektronik, sehingga seharusnya data yang paling terbanyaka adalah cuata cerah. Untuk memvalidasi hal tersebut dilakukan validasi menggunakan GX.

In [16]:
# ensure weather condition's mode is sunny
validator.expect_column_most_common_value_to_be_in_set(
    'weather_condition',
    ['sunny']
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "sunny"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Tervalidasi bahwa 'weather_condition' terbanyak adalah sunny.

# Result

In [17]:
# save result
valid_res = validator.validate()

# Extract results for each expectation
result_rows = []
for result in valid_res["results"]:
    # dataframe value
    row = {
        "expectation_type": result["expectation_config"]["expectation_type"],
        "column": result["expectation_config"]["kwargs"].get("column"),
        "success": result["success"]
    }

    # add to list
    result_rows.append(row)

# Create DataFrame from results
results_df = pd.DataFrame(result_rows)

# show result
results_df

Calculating Metrics:   0%|          | 0/65 [00:00<?, ?it/s]

,expectation_type,column,success
0,expect_column_values_to_be_unique,code,True
1,expect_column_values_to_be_between,discount,True
2,expect_column_values_to_be_between,inventory_level,True
3,expect_column_values_to_be_between,units_sold,True
4,expect_column_values_to_be_between,units_ordered,True
5,expect_column_values_to_be_between,price,True
6,expect_column_values_to_be_in_type_list,price,True
7,expect_column_values_to_be_between,competitor_pricing,True
8,expect_column_values_to_be_in_type_list,competitor_pricing,True
9,expect_column_values_to_be_in_set,weather_condition,True


Secara keseluruhan, semua validasi berhasil dilakukan dengan hasil yang memuaskan. Data ini sekarang dapat dianggap siap untuk analisis lebih lanjut atau pemrosesan tambahan.